# DiffiMax: The "Chess Trap Finder" Prototype
## A more human-useful version of Minimax

## This algorithm is essentially Minimax, but it chooses the second-best option for your opponent if there is enough of an evaluation difference between your opponent's first and second-best options and there are a substantial number of move choices on your opponent's turn. That is, it prioritizes traps by assuming that your opponent will make a mistake but that, if they get it correct, your evaluation is still in the "drawish range."

## There are other conditions that can be added for what constitutes a trap candidate, like weighing the trap evaluation difference based on the number of moves in the position.

In [1]:
import random

class createTree:
    def __init__(self,depth,player=1):
        self.depth = depth
        self.player = player
        self.value = None
        self.capture = random.randint(1,5) # the value 1 represents a capture
        self.check = random.randint(1,5) # the value 1 represents a check
        self.children = []
        self.childVals()
        
    def childVals(self):
        if self.depth > 0:
            self.branchCount = random.randint(1,30) # Range of possible branches
            # print(self.depth,self.branchCount) # Prints out number of tree branches
            for i in range(0,self.branchCount):
                self.children.append(createTree(self.depth-1,player=-self.player))
        elif self.depth == 0:
            self.value = random.gauss(.1,1.5) # Mean and SD of leaf position evaluations

In [7]:
def DiffiMax(node,drawThresh,winThresh,yourNum):
    if node.depth != 1: # Cycles through all nodes
        for child in node.children:
            child.value = DiffiMax(child,drawThresh,winThresh,yourNum)
        node.value = DiffiSort(node,drawThresh,winThresh,yourNum)
        #print('Depth:',node.depth)
        #print('Value:',node.value)
        return node.value
    else: # Nodes with depth 1 start here and return 
        depth1Val = DiffiSort(node,drawThresh,winThresh,yourNum)
        return depth1Val

def DiffiSort(node,drawThresh,winThresh,yourNum):
    node.children.sort(reverse=True,key=lambda a:a.value) # Sorts children for future selections
    if node.player == -yourNum: # If it is your opponent's choice
        if len(node.children) == 1: # If node has one child, as next condition requires two children.
            return node.children[0].value
        elif node.player == -1: # Opponent has black pieces
            if (node.children[-1].value - node.children[-2].value <= -winThresh)\
                                        & (node.children[-2].value >= winThresh)\
                                        & (len(node.children)>=10)\
                                        & (node.children[-1].value >= -drawThresh)\
                                        & (node.check != 1) & (node.capture != 1)\
                                        & (node.children[-1].capture != 1)\
                                        & (node.children[-2].capture != 1):
                                        # Conditions in order include the following:
                                        # Does the evaluation difference make it a trap?
                                        # Are you now winning?
                                        # Are there more than 10 moves in the position?
                                        # Is opponent's best move still a draw for you?
                                        # Are current/previous positions checks/captures?
                print('Trap Found!')
                print('Depth:',node.children[-2].depth)
                print('Evaluation:',node.children[-2].value)
                return node.children[-2].value
            else:
                return node.children[-1].value
        elif node.player == 1: # Opponent has white pieces
            if (node.children[0].value - node.children[1].value >= winThresh)\
                                        & (node.children[1].value <= -winThresh)\
                                        & (len(node.children)>=10)\
                                        & (node.children[0].value <= drawThresh)\
                                        & (node.check != 1) & (node.capture != 1)\
                                        & (node.children[0].capture != 1)\
                                        & (node.children[1].capture != 1):
                print('Trap Found!')
                print('Depth:',node.children[1].depth)
                print('Evaluation:',node.children[1].value)
                return node.children[1].value
            else:
                return node.children[0].value
    elif node.player == yourNum: # If it is your choice
        if node.player == 1: # You have the white pieces
            return node.children[0].value
        
        elif node.player == -1: # You have the black pieces
            return node.children[-1].value

In [8]:
tree = createTree(5)
# If depth == even, algorithm begins with opponent choice
# (because choice starts with depth 1)

In [9]:
# DiffiMax(node,drawThresh,winThresh,yourNum)

# drawThresh is the evaluation that is still a draw (always positive)
# winThresh is the evaluation that is considered winning (always positive)
# yourNum is the color of "player": 1 for white, -1 for black

Evaluation = DiffiMax(tree,1,2,1)

Trap Found!
Depth: 1
Evaluation: 2.0821885180961703
Trap Found!
Depth: 1
Evaluation: 2.0009887436559817


In [11]:
print('Final Evaluation:',Evaluation)

Final Evaluation: 1.620861579762952
